<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/TrackNet_minVIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Mon Aug 22 11:08:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Inicializacion

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import os 
os.chdir('/content/')

!git clone https://github.com/VictorUceda/TrackNet-Padel.git
!git clone https://github.com/NVlabs/MinVIS.git
#!git clone https://github.com/Chang-Chia-Chi/TrackNet-Badminton-Tracking-tensorflow2
os.chdir('/content/TrackNet-Padel/')
!git pull
!git checkout -- /content/TrackNet-Padel/points/cleanVal/*

Cloning into 'TrackNet-Padel'...
remote: Enumerating objects: 1201, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 1201 (delta 60), reused 54 (delta 21), pack-reused 1073
Receiving objects: 100% (1201/1201), 1.08 GiB | 24.31 MiB/s, done.
Resolving deltas: 100% (527/527), done.
Checking out files: 100% (329/329), done.
Cloning into 'MinVIS'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 144 (delta 39), reused 144 (delta 39), pack-reused 0
Receiving objects: 100% (144/144), 152.07 KiB | 1.54 MiB/s, done.
Resolving deltas: 100% (39/39), done.
Already up to date.
error: pathspec '/content/TrackNet-Padel/points/cleanVal/*' did not match any file(s) known to git.


In [30]:
os.chdir('/content/MinVIS/')

In [31]:
!python -m pip install pyyaml==5.1
#!pip install 'git+https://github.com/facebookresearch/detectron2.git@5aeb252b194b93dc2879b4ac34bc51a31b5aee13'
!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2
!git checkout 5aeb252b194b93dc2879b4ac34bc51a31b5aee13
%cd ..
!python -m pip install -e detectron2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'detectron2'...
remote: Enumerating objects: 14506, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 14506 (delta 29), reused 54 (delta 19), pack-reused 14425
Receiving objects: 100% (14506/14506), 5.96 MiB | 7.84 MiB/s, done.
Resolving deltas: 100% (10470/10470), done.
/content/MinVIS/detectron2
Note: checking out '5aeb252b194b93dc2879b4ac34bc51a31b5aee13'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 5aeb252 upgrade pyre version in `fbcode/vi

In [32]:
import torch, detectron2
!nvcc --version
TORCH_VERSION= ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION= detectron2.__version__.split(".")[-1]
print("torch: ",TORCH_VERSION, "; cuda: ",CUDA_VERSION)
print("detectron: ",detectron2.__version__)


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
torch:  1.12 ; cuda:  6
detectron:  0.6


In [ ]:
os.chdir('/content/MinVIS/')
!pip install -r requirements.txt
os.chdir('/content/MinVIS/mask2former/modeling/pixel_decoder/ops/')
!sh make.sh

In [36]:
os.chdir('/content/MinVIS/demo_video/')
#!gdown https://drive.google.com/u/0/uc?id=12yL72Qv8OBqapgvGmLHXAb6YZv4SS_6D #minvis_ytvis19_swin_large.pth
!gdown https://drive.google.com/u/0/uc?id=1mdrL6QRVmoz-QizohZ7SnyGDlpAWFCVf #minvis_ytvis19_R50.pth


Downloading...
From: https://drive.google.com/u/0/uc?id=1mdrL6QRVmoz-QizohZ7SnyGDlpAWFCVf
To: /content/MinVIS/demo_video/minvis_ytvis19_R50.pth
100% 528M/528M [00:05<00:00, 97.0MB/s]


In [37]:
os.chdir('/content/MinVIS/demo_video/')
!python demo.py --config-file ../configs/youtubevis_2019/video_maskformer2_R50_bs32_8ep_frame.yaml \
  --input /content/TrackNet-Padel/train_data/match1/x_data \
  --output /content/MinVIS/output \
  --opts MODEL.WEIGHTS minvis_ytvis19_R50.pth

[08/22 12:21:01 detectron2]: Arguments: Namespace(confidence_threshold=0.5, config_file='../configs/youtubevis_2019/video_maskformer2_R50_bs32_8ep_frame.yaml', input='/content/TrackNet-Padel/train_data/match1/x_data', opts=['MODEL.WEIGHTS', 'minvis_ytvis19_R50.pth'], output='/content/MinVIS/output')
[08/22 12:21:04 fvcore.common.checkpoint]: [Checkpointer] Loading from minvis_ytvis19_R50.pth ...
[08/22 12:21:04 d2.checkpoint.c2_model_loading]: Following weights matched with model:
| Names in Model                                                               | Names in Checkpoint                                                                                                                                                                                                                                                                                                                                                                                                                              

## Video2frames

In [19]:
os.chdir('/content/TrackNet-Padel/')
!mv raw_data/*.* raw_data/points/
for point_n in range(9):
  files = 'raw_data/points/clean/*_'+str(point_n)+'\.*'
  print(files)
  !mv {files} raw_data/

raw_data/points/clean/*_0\.*
raw_data/points/clean/*_1\.*
raw_data/points/clean/*_2\.*
raw_data/points/clean/*_3\.*
raw_data/points/clean/*_4\.*
raw_data/points/clean/*_5\.*
raw_data/points/clean/*_6\.*
raw_data/points/clean/*_7\.*
raw_data/points/clean/*_8\.*


In [20]:
!rm -rf train_data/*
!python video2img.py --HEIGHT=480 --WIDTH=720 --sigma=1

['raw_data/PremierRoma_Octavos_GRMS_2_53_0.mp4', 'raw_data/PremierRoma_Octavos_GRMS_2_53_1.mp4', 'raw_data/PremierRoma_Octavos_GRMS_2_53_2.mp4', 'raw_data/PremierRoma_Octavos_GRMS_2_53_5.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_00_0.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_00_1.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_00_2.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_00_3.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_00_4.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_23_1.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_23_2.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_23_3.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_23_4.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_23_5.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_44_1.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_44_2.mp4', 'raw_data/PremierRoma_Semis_LGBC_0_44_3.mp4', 'raw_data/PremierRoma_Semis_LGBC_1_00_0.mp4', 'raw_data/PremierRoma_Semis_LGBC_1_00_1.mp4', 'raw_data/PremierRoma_Semis_LGBC_1_00_2.mp4', 'raw_data/PremierRoma_Semis_LGBC_1_00_4.mp4', 'raw_data/PremierRoma_Sem